In [14]:
import numpy as np
import pandas as pd

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [16]:
df = pd.read_csv('covid_toy.csv')

In [17]:
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [18]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [19]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [21]:
X_train

,age,gender,fever,cough,city
32,34,Female,101.0,Strong,Delhi
0,60,Male,103.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata
4,65,Female,101.0,Mild,Mumbai
2,42,Male,101.0,Mild,Delhi
...,...,...,...,...,...
18,64,Female,98.0,Mild,Bangalore
91,38,Male,NaN,Mild,Delhi
78,11,Male,100.0,Mild,Bangalore
72,83,Female,101.0,Mild,Kolkata


In [22]:
y_test

41    Yes
61     No
5     Yes
47     No
95     No
15    Yes
93    Yes
6      No
96    Yes
34    Yes
94    Yes
82    Yes
75    Yes
80    Yes
49     No
46     No
79    Yes
55    Yes
54    Yes
60    Yes
Name: has_covid, dtype: object

# 1- Normal way

In [23]:
# simple imputer  to fever colum
# imputer is used for the fill the missing values 

si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# aslo the test data
X_test_fever = si.transform(X_test[['fever']])

In [24]:
X_train_fever.shape

(80, 1)

In [25]:
# Ordinal Encoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# aslo for the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [27]:
# One Hot Encoding  -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

#also for the test data
X_test_gender_city = ohe.transform(X_test[['gender','city']])

X_train_gender_city.shape


C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [28]:
# Extracting the Age 
X_train_Age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_Age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_Age.shape

(80, 1)

In [30]:
X_train_Age

array([[34],
       [60],
       [69],
       [65],
       [42],
       [26],
       [80],
       [34],
       [68],
       [75],
       [31],
       [16],
       [34],
       [22],
       [38],
       [65],
       [20],
       [83],
       [64],
       [12],
       [15],
       [47],
       [23],
       [55],
       [10],
       [40],
       [59],
       [19],
       [82],
       [42],
       [27],
       [20],
       [49],
       [42],
       [33],
       [20],
       [73],
       [65],
       [19],
       [17],
       [83],
       [72],
       [47],
       [49],
       [75],
       [25],
       [80],
       [64],
       [82],
       [ 5],
       [11],
       [51],
       [46],
       [71],
       [23],
       [34],
       [69],
       [51],
       [19],
       [71],
       [65],
       [66],
       [54],
       [73],
       [ 8],
       [10],
       [ 6],
       [13],
       [50],
       [16],
       [25],
       [27],
       [49],
       [56],
       [ 5],
       [64],
       [38],

In [32]:
X_train_transformed = np.concatenate((X_train_Age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_Age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [34]:
X_test_transformed.shape

(20, 7)

# Mentos Zindghi

In [35]:
from sklearn.compose import ColumnTransformer


In [36]:
# remainder = drop => drop the remaining columns
# remainder = passthrough => remains same the other column

transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [37]:
transformer.fit_transform(X_train)

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [103.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  60.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  65.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  80.        ],
       [104.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  34.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.    

In [39]:
transformer.fit_transform(X_train).shape

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [40]:
transformer.fit_transform(X_test)

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[101.29411765,   0.        ,   1.        ,   1.        ,
          0.        ,  82.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,  81.        ],
       [101.29411765,   0.        ,   0.        ,   0.        ,
          0.        ,  84.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  18.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  12.        ],
       [103.        ,   1.        ,   1.        ,   1.        ,
          0.        ,  70.        ],
       [100.        ,   0.        ,   1.        ,   1.        ,
          0.        ,  27.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,  14.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,  51.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,  74. 